## Homework

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the
[previous homework](../08-deep-learning/homework.md).

Download the model files from here: 

* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
* https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

With wget:

```bash
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL="${PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL="${PREFIX}/hair_classifier_v1.onnx"
wget ${DATA_URL}
wget ${MODEL_URL}
```

In [2]:
# Run once

PREFIX = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL = f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL = f"{PREFIX}/hair_classifier_v1.onnx"

# !wget {DATA_URL}
# !wget {MODEL_URL}

--2025-12-08 12:04:01--  https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/426348925/398ded4a-c41c-4e5a-9672-acb7e441de54?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-08T11%3A38%3A17Z&rscd=attachment%3B+filename%3Dhair_classifier_v1.onnx.data&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-08T10%3A37%3A19Z&ske=2025-12-08T11%3A38%3A17Z&sks=b&skv=2018-11-09&sig=RNhucEcFzS2in6KCBrGDJsH%2BlmGa58mFXYSBxYqCv%2Bs%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTE5MzY0MiwibmJmIjoxNzY1MTkxODQyLCJ

## Question 1

To be able to use this model, we need to know the name of the input and output nodes. 

What's the name of the output:

* `output`
* `sigmoid`
* `softmax`
* `prediction`

In [40]:
import onnx

# Load the ONNX model
model = onnx.load("hair_classifier_v1.onnx")

# Print the model's inputs and outputs
print("Inputs:", [input.name for input in model.graph.input])
print("Outputs:", [output.name for output in model.graph.output])

Inputs: ['input']
Outputs: ['output']


## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

In [ ]:
from io import BytesIO
from urllib import request

from PIL import Image


def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != "RGB":
        img = img.convert("RGB")
    img = img.resize(target_size, Image.NEAREST)
    return img

## Question 2: Target size

Let's download and resize this image: 

https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

Based on the previous homework, what should be the target size for the image?

* 64x64
* 128x128
* **200x200**
* 256x256

In [ ]:
img_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img_raw = download_image(img_url)
img = prepare_image(img_raw, [200, 200])

In [43]:
width, height = img.size

print(f"Width: {width}, Height: {height}")

Width: 200, Height: 200


## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

Old code was like this

In [44]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_transforms = transforms.Compose(
    [
        transforms.Resize((200, 200)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # ImageNet normalization
    ]
)

In [45]:
from torch import Tensor

# Apply the transforms
img_tensor: Tensor = train_transforms(img)

# The first pixel's R channel value
first_pixel_r = img_tensor[0, 0, 0].item()

print(f"First pixel, R channel: {first_pixel_r}")

First pixel, R channel: -1.0732940435409546


After the pre-processing, what's the value in the first pixel, the R channel?

* -10.73
* **-1.073**
* 1.073
* 10.73

In [62]:
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image

# ImageNet normalization constants
MEAN = np.array([0.485, 0.456, 0.406], dtype="float32")
STD = np.array([0.229, 0.224, 0.225], dtype="float32")


def train_transform_small(img):
    img_array = (
        np.array(img).astype("float32") / 255.0
    )  # PIL to NumPy, normalize to [0, 1]
    img_array = (img_array - MEAN) / STD  # Normalize with ImageNet stats
    img_array = np.transpose(img_array, (2, 0, 1))  # HWC to CHW (like ToTensor)
    return img_array

In [63]:
# Apply the transforms
img_tensor_2: np.ndarray = train_transform_small(img)

# The first pixel's R channel value
first_pixel_r_2 = img_tensor_2[0, 0, 0].item()

print(f"First pixel, R channel: {first_pixel_r_2}")

First pixel, R channel: -1.0732940435409546


## Question 4

Now let's apply this model to this image.

In [64]:
import onnxruntime as ort

input_data = img_tensor.unsqueeze(0).numpy()

print(type(input_data))
print(input_data.shape)
print(input_data.dtype)
sess = ort.InferenceSession("hair_classifier_v1.onnx")
print(sess.run(None, {"input": input_data}))


input_data_2 = np.expand_dims(img_tensor_2, 0)
print(type(input_data_2))
print(input_data_2.shape)
print(input_data_2.dtype)
print(sess.run(None, {"input": input_data_2}))

<class 'numpy.ndarray'>
(1, 3, 200, 200)
float32
[array([[0.09156641]], dtype=float32)]
<class 'numpy.ndarray'>
(1, 3, 200, 200)
float32
[array([[0.09156641]], dtype=float32)]


 What's the output of the model?

* **0.09**
* 0.49
* 0.69
* 0.89

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  

## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.13

COPY hair_classifier_empty.onnx.data .
COPY hair_classifier_empty.onnx .
```

Note that it uses Python 3.13.

The docker image is published to [`agrigorev/model-2025-hairstyle:v1`](https://hub.docker.com/r/agrigorev/model-2025-hairstyle).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.

## Question 5

Download the base image `agrigorev/model-2025-hairstyle:v1`. You can do it with [`docker pull`](https://docs.docker.com/engine/reference/commandline/pull/).

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [29]:
! docker pull agrigorev/model-2025-hairstyle:v1

v1: Pulling from agrigorev/model-2025-hairstyle

54c34aa3: Pulling fs layer 
9533db7f: Pulling fs layer 
d8a1e1c2: Pulling fs layer 
ecca3b37: Pulling fs layer 
94d707b7: Pulling fs layer 
46295de2: Pulling fs layer 
a27bb275: Pulling fs layer 
Digest: sha256:9e43d5a5323f7f07688c0765d3c0137af66d0154af37833ed721d6b4de6df528
Status: Downloaded newer image for agrigorev/model-2025-hairstyle:v1
docker.io/agrigorev/model-2025-hairstyle:v1


In [30]:
! docker images

REPOSITORY                       TAG       IMAGE ID       CREATED        SIZE
agrigorev/model-2025-hairstyle   v1        4528ad1525d5   6 days ago     608MB
income_prediction_model          latest    6f1a47ee9524   2 weeks ago    2.67GB
streamlit_image                  latest    30cd1d151caf   2 weeks ago    1.43GB
diabetes-service                 latest    15265e4b836d   2 weeks ago    1.11GB
julxi/ml-zoomcamp-midterm        2025      07ec014b5b2b   2 weeks ago    520MB
julxi/ml-zoomcamp-midterm        <none>    f7c4c025b4d3   2 weeks ago    460MB
<none>                           <none>    9b8cbe6364c2   2 weeks ago    212MB
ml-zoomcamp-homework-05          2025      1f1fe8338dc7   6 weeks ago    204MB
agrigorev/zoomcamp-model         2025      4a9ecc576ae9   6 weeks ago    121MB
hello-world                      latest    1b44b5a3e06a   4 months ago   10.1kB


So what's the size of this base image?

* 88 Mb
* 208 Mb
* **608 Mb**
* 1208 Mb

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `hair_classifier_empty.onnx` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg

What's the output from the model?

* -1.0
* **-0.10**
* 0.10
* 1.0

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


## Submit the results

* Submit your results here: https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw09
* If your answer doesn't match options exactly, select the closest one. If the answer is exactly in between two options, select the higher value.

## Publishing to Docker hub

Just for the reference, this is how we published our image to Docker hub:

```bash
docker build -t model-2025-hairstyle -f homework.dockerfile .
docker tag model-2025-hairstyle:latest agrigorev/model-2025-hairstyle:v1
docker push agrigorev/model-2025-hairstyle:v1
```

(You don't need to execute this code)